In [ ]:
# visualization of simulation data

# Jan Totz <jantotz@itp.tu-berlin.de>

In [ ]:
# dependencies
import numpy as np
import os.path
import getpass
import matplotlib.pyplot as plt
import matplotlib as mpl
from ipywidgets import widgets, interact

In [ ]:
def loadData(pth):

    # get pth and name of file
    pthfn = os.path.join(pth, 'states', 'state.bin')
    if not os.path.isfile(pthfn):
        print('File does not exist!'); quit();
    
    # define header datatype
    headerDatatypes = np.dtype([
    ("nx", np.int32),
    ("nc", np.int32),
    ("dt", np.float32),
    ])
    
    # load data
    f = open(pthfn, "rb")                                                           # open the file
    [nx, nc, dt] = np.fromfile(f, dtype=headerDatatypes, count=1)[0]                # read header
    rawData = np.fromfile(f, dtype=np.float64)                                      # read body
    f.close()                                                                       # close the file                               
    
    return [rawData, nc, dt, nx]

In [ ]:
def prepareData(rawData):
    
    # wrap data of phase oscillator to [0,2pi[
    if(phaseOsciQ):
        rawData = rawData % 2*np.pi
    
    shapedData = np.reshape(rawData, (-1,ncomponents,nx))                           # reshape array for future use
    nstates = shapedData.shape[0]                                                   # find number of states

    return [shapedData, nstates]

In [ ]:
def plot_frame_nw(t):
    mpl.rcParams['xtick.labelsize'] = 18
    mpl.rcParams['ytick.labelsize'] = 18
    fsAxesLabels = 20
    fsPlotLabels = 22
    fsMainLabel = 24

    fig = plt.figure(figsize=(16,5.5))
    fig.suptitle('t = ' + '%.1f' % t, fontsize=fsMainLabel)

    if(ncomponents == 1):
        # data
        x = np.arange(1,nx+1)
        y = uvData[t,0,:]
        # plot
        plt.scatter(x, y)
        # plot options
        plt.title('u',{'fontsize':fsPlotLabels})
        plt.xlabel('index i',{'fontsize':fsAxesLabels})
        plt.ylabel('variable u',{'fontsize':fsAxesLabels})
        
    elif(ncomponents == 2):
        # data
        x = np.arange(1,nx+1)
        y1 = uvData[t,0,:]
        y2 = uvData[t,1,:]
        # plot
        plt.scatter(x,y1, c='b', marker='x', label='1')
        plt.scatter(x,y2, c='r', marker='s', label='-1')
        # plot options
        plt.title('u,v',{'fontsize':fsPlotLabels})
        plt.xlabel('index i',{'fontsize':fsAxesLabels})
        plt.ylabel('variables u,v',{'fontsize':fsAxesLabels})
        
    plt.show()

In [ ]:
def plot_frame_1d(t):
    mpl.rcParams['xtick.labelsize'] = 18
    mpl.rcParams['ytick.labelsize'] = 18
    fsAxesLabels = 20
    fsPlotLabels = 22
    fsMainLabel = 24

    fig = plt.figure(figsize=(16,5.5))
    fig.suptitle('t = ' + '%.1f' % t, fontsize=fsMainLabel)

    if(ncomponents == 1):
        # data
        x = np.arange(1,nx+1)
        y = uvData[t,0,:]
        # plot
        plt.scatter(x, y)
        # plot options
        plt.title('u',{'fontsize':fsPlotLabels})
        plt.xlabel('index i',{'fontsize':fsAxesLabels})
        plt.ylabel('variable u',{'fontsize':fsAxesLabels})
        plt.axis(plotRanges)
        
    elif(ncomponents == 2):
        # data
        x = np.arange(1,nx+1)
        y1 = uvData[t,0,:]
        y2 = uvData[t,1,:]
        # plot
        plt.scatter(x,y1, c='b', marker='x', label='1')
        plt.scatter(x,y2, c='r', marker='s', label='-1')
        # plot options
        plt.title('u,v',{'fontsize':fsPlotLabels})
        plt.xlabel('index i',{'fontsize':fsAxesLabels})
        plt.ylabel('variables u,v',{'fontsize':fsAxesLabels})
        plt.axis(plotRanges)
        
    plt.show()

In [ ]:
def plot_frame(t):
    if(networkQ):
        plot_frame_nw(t)
    else:
        plot_frame_1d(t)

In [ ]:
# main
pth = "/home/"+getpass.getuser()+"/Desktop/Simulations/phase_nonlocal_cpu";
[rawData, ncomponents, dt, nx] = loadData(pth)

# model options
phaseOsciQ=True
[uvData, nstates] = prepareData(rawData)

# plot options
networkQ=False
plotRanges = [0, nx+1, 0, 2*np.pi]     # [xmin,xmax,ymin,ymax]

In [ ]:
# choose specific frame
interact(plot_frame_1d, t=widgets.IntSlider(min=0,max=nstates-1,step=1,value=0));

In [ ]:
# animation
interact(plot_frame, t=widgets.Play(min=0,max=nstates-1,step=1,value=0));

In [ ]:
# space time plot
variableIndex=0
dn=100                                             # leave out some states for faster plotting

x_grid=np.linspace(0,nstates*dt,nstates)[::dn]
y_grid=np.arange(1,nx+1)
x_mesh,y_mesh=np.meshgrid(x_grid,y_grid)
mpl.rcParams['xtick.labelsize'] = 18
mpl.rcParams['ytick.labelsize'] = 18
fsAxesLabels = 20
fsPlotLabels = 22
fsMainLabel = 24
plt.pcolormesh(x_mesh,y_mesh,uvData[::dn,variableIndex,:].T)
plt.xlabel('time t',{'fontsize':fsAxesLabels})
plt.ylabel('index i',{'fontsize':fsAxesLabels})
colorbar = plt.colorbar(fraction=0.046, pad=0.04)
plt.clim(plotRanges[2:])
colorbar.set_label('u',size=fsAxesLabels)
plt.show()